In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
import math
from IPython.display import FileLink


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
    #for filename in filenames: print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames: 
        print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

/kaggle/input
/kaggle/input/hf-round2-train
/kaggle/input/hf-round2-train/hf_round2_train
/kaggle/input/hf-round2-train/hf_round2_train/hf_round2_train
/kaggle/input/prepare-data
/kaggle/input/ecgdata
/kaggle/input/ecgdata/hf_round1_train
/kaggle/input/ecgdata/hf_round1_testA
/kaggle/working/__notebook__.ipynb


In [2]:
!cp ../input/prepare-data/disease_dict.pickle .

from random import shuffle
import gzip
round2_add = '../input/prepare-data/tianchi_round2.gz'
round2_f = gzip.open(round2_add,'rt')
datas = []
for line in round2_f:
    datas.append(line)

shuffle(datas)

f = gzip.open('tianchi_round2.gz','wt+')
#f = open('tianchi_round2','w+')
for line in tqdm(datas):
    f.write(line)
    if line[-1] != '\n':
        f.write('\n')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


In [3]:
"""
#load diseases dict
import pickle
def load_disease_dict(path):
    disease_dict = {}
    with open(path,'r') as dict_f:
        for count, line in enumerate(dict_f):
            disease_dict[line.replace('\n','')] = count
    return disease_dict
disease_dict = load_disease_dict('/kaggle/input/ecgdata/hf_round1_arrythmia.txt')
pickle.dump(disease_dict,open('disease_dict.pickle','wb+'))
disease_dict

def load_labels(path):
    #load labels
    labels = []
    with open(path,'r') as f:
        for count, line in tqdm(enumerate(f)):
            datas = line.split()
            diseases = []
            exist_age = 0
            exist_sex = 0
            if datas[1].isdigit():  exist_age = 1
            if datas[1] == 'MALE' or datas[1] == 'FEMALE' : exist_sex = 1
            if len(datas) > 2 and (datas[2] == 'MALE' or datas[2] == 'FEMALE') : exist_sex = 1
            for disease in datas[1+exist_age+exist_sex:]:
                diseases.append(disease_dict[disease])
            labels.append(diseases)

    #transfer labels to one-hot
    hot_labels = []
    for label in labels:
        hot = [0]*len(disease_dict)
        for disease in label:
            hot[disease] = 1
        hot_labels.append(hot)
    hot_labels = torch.FloatTensor(hot_labels)
    hot_labels.shape

    num_dict = {}
    for dis in labels:
        for di in dis:
            num_dict[di] = num_dict.get(di,0) + 1
    #dict(sorted(num_dict.items()))
    return labels, hot_labels, num_dict
labels, hot_labels, num_dict = load_labels('/kaggle/input/ecgdata/hf_round1_label.txt')
#labels, hot_labels, num_dict = load_labels('/kaggle/input/hf-round2-train/hf_round2_train.txt')
num_dict

'''train_data = torch.load('/kaggle/input/hf-round2-train/train_data.tensor')
hot_labels = torch.load('/kaggle/input/hf-round2-train/hot_labels.tensor')
train_data.shape'''

def get_train_data(path):
    train_data = []
    for dirname, dirs, filenames in os.walk(path):
        #filenames = sorted(filenames)
        filenames.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        for file_count, filename in tqdm(enumerate(filenames)):
            address = os.path.join(dirname, filename)
            #if file_count == 10000: break
            with open(address,'r') as f:
                individual = []
                for line, v in enumerate(f):
                    if line == 0: continue
                    v = v.split()
                    #print(v)
                    individual.append([float(value) for value in v])
                    del v
                individual = torch.FloatTensor(individual).transpose(0,1)
                #train_data[file_count] = individual
                train_data.append(individual)
                del individual
                f.close()
            #break
    train_data = torch.stack(train_data)
    return train_data
#get_train_data('/kaggle/input/hf-round2-train/hf_round2_train')
#train_data = get_train_data('/kaggle/input/hf-round2-train/hf_round2_train')
train_data = get_train_data('/kaggle/input/ecgdata/hf_round1_train')

#add up to 12 leads
do_12leads = 1
if do_12leads:
    I = train_data[:,0]
    II = train_data[:,1]
    III=II-I
    aVR=-(I+II)/2
    aVL=I-II/2
    aVF=II-I/2

    III = III.unsqueeze(1)
    aVR = aVR.unsqueeze(1)
    aVL = aVL.unsqueeze(1)
    aVF = aVF.unsqueeze(1)
    train_data = torch.cat([train_data, III, aVR, aVL, aVF],dim=1)
    del I, II, III, aVR, aVL, aVF
train_data.shape

import base64
def write_data_to_file(output_path, datas, labels):
    key = 0
    with open(output_path,'w+') as f:
        for data,label in zip(datas,labels):
            output_str = [str(key), base64.b64encode(data.numpy().astype(np.float16)).decode("utf-8"),
                              base64.b64encode(label.numpy().astype(np.float16)).decode("utf-8"), '-1']
            f.write('\t'.join(output_str))
            #if key != 20035:
            if key != 24105:
                f.write('\n')
            key += 1
            #print(key)
write_data_to_file('tianchi_round2', train_data, hot_labels)

!tar -zcf tianchi_round2.gz tianchi_round2
!rm tianchi_round2
#exit(0)"""

'\n#load diseases dict\nimport pickle\ndef load_disease_dict(path):\n    disease_dict = {}\n    with open(path,\'r\') as dict_f:\n        for count, line in enumerate(dict_f):\n            disease_dict[line.replace(\'\n\',\'\')] = count\n    return disease_dict\ndisease_dict = load_disease_dict(\'/kaggle/input/ecgdata/hf_round1_arrythmia.txt\')\npickle.dump(disease_dict,open(\'disease_dict.pickle\',\'wb+\'))\ndisease_dict\n\ndef load_labels(path):\n    #load labels\n    labels = []\n    with open(path,\'r\') as f:\n        for count, line in tqdm(enumerate(f)):\n            datas = line.split()\n            diseases = []\n            exist_age = 0\n            exist_sex = 0\n            if datas[1].isdigit():  exist_age = 1\n            if datas[1] == \'MALE\' or datas[1] == \'FEMALE\' : exist_sex = 1\n            if len(datas) > 2 and (datas[2] == \'MALE\' or datas[2] == \'FEMALE\') : exist_sex = 1\n            for disease in datas[1+exist_age+exist_sex:]:\n                diseases.ap